In [1]:
%load_ext lab_black

Step: Load data

In [2]:
import csv
import pandas as pd
import numpy as np
import time
import os
from pyprojroot import here
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import median_absolute_error
from math import sqrt

path_data = here("./data")
os.chdir(path_data)

data_draft_kings_analysis_week = pd.read_parquet(
    "data_draft_kings_analysis_week.parquet"
)

In [3]:
data_draft_kings_analysis_week["points"].describe()

count    20.000000
mean     25.191537
std       2.043255
min      22.076296
25%      23.669537
50%      24.852963
75%      26.337593
max      29.214815
Name: points, dtype: float64

In [6]:
import pandas as pd
from scipy.stats import spearmanr


def filter_correlations(data, outcome, threshold):
    # Calculate the Spearman correlation
    correlation_matrix, _ = spearmanr(data)

    # Store the correlations in a new Series
    correlations = pd.Series(
        correlation_matrix[data.columns.get_loc(outcome)], index=data.columns
    )

    # Filter the correlations, removing the correlation of the outcome variable with itself
    correlations = correlations.drop(outcome)

    # Create a DataFrame with the correlations and predictor names
    correlations_df = (
        pd.DataFrame(correlations)
        .reset_index()
        .rename(columns={0: "correlation", "index": "predictor"})
    )

    # Filter the DataFrame based on the correlation threshold
    correlations_df["abs_corr"] = correlations_df["correlation"].abs()
    filtered_correlations = (
        correlations_df[correlations_df["abs_corr"] > threshold]
        .sort_values(by="abs_corr", ascending=False)
        .reset_index(drop=True)
        .drop(columns=["abs_corr"])
    ).round(2)

    return filtered_correlations

In [9]:
data = data_draft_kings_analysis_week
outcome = "points"
threshold = 0.40

filtered_correlations = filter_correlations(data, outcome, threshold)
filtered_correlations.to_csv("filtered_correlations.csv")
filtered_correlations

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,predictor,correlation
0,value,0.88
1,sony_michel,0.54
2,david_montgomery,0.52
3,antonio_brown,0.52
4,braxton_berrios,0.49
5,josh_allen,-0.45
6,patrick_mahomes,-0.41
7,tom_brady,0.41
